In [1]:
import os

os.chdir('../..')

In [2]:
import glob
import os
import re
from collections import OrderedDict

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.ndimage.measurements import center_of_mass

import analysis.images as im
import modules.numpy_funcs as nf
import modules.point_processing as pp

In [3]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [4]:
load_dir = os.path.join('data', 'kinect', 'labelled_trials')

best_pos_dir = os.path.join('data', 'kinect', 'best_pos')

In [5]:
labelled_trial_names = os.listdir(load_dir)

In [6]:
body_part_grayscale = OrderedDict({'HEAD': 105,
                       'L_HIP': 93,
                       'R_HIP': 40,
                       'L_THIGH': 133, 
                       'R_THIGH': 240,
                       'L_KNEE': 52,
                       'R_KNEE': 111,
                       'L_CALF': 36,
                       'R_CALF': 37,
                       'L_FOOT': 135,
                       'R_FOOT': 76,
                      })


In [7]:
best_pos_dir = os.path.join('data', 'kinect', 'best_pos')


# Camera calibration parameters
x_res, y_res = 565, 430
f_xz, f_yz = 1.11146664619446, 0.833599984645844


pattern = re.compile(r'(\d+)\.png')

for trial_name in labelled_trial_names[:1]:
    
    # Best positions selected by our method
    df_best_pos = pd.read_pickle(os.path.join(best_pos_dir, trial_name + '.pkl'))
    
    label_dir = os.path.join(load_dir, trial_name, 'label')
    depth_dir = os.path.join(load_dir, trial_name, 'depth16bit')

    df_estimated = pd.read_pickle(os.path.join(best_pos_dir, trial_name + '.pkl'))
    
    label_paths = sorted(glob.glob(os.path.join(label_dir, '*.png')))
    depth_paths = sorted(glob.glob(os.path.join(depth_dir, '*.png')))
    
    depth_filenames = [os.path.basename(x) for x in depth_paths]
    frames = [int(re.search(pattern, x).group(1)) for x in depth_filenames]
    
    for i, frame in enumerate(frames):

        
        label_path, depth_path = label_paths[i], depth_paths[i]
                        
        label_image = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        depth_image = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)
        
        binary_image = label_image == body_part_grayscale['HEAD']
        
        if np.any(binary_image):
        
            y, x = np.round(center_of_mass(binary_image)).astype(int)
            z = depth_image[y, x] / 10  # Convert mm to cm
            
            point_real = im.image_to_real([x, y, z], x_res, y_res, f_xz, f_yz)
            
                
        if frame == 295:
            break

In [8]:
label_filenames = [os.path.basename(x) for x in label_paths]

depth_frames = [int(re.search(pattern, x).group(1)) for x in depth_filenames]
label_frames = [int(re.search(pattern, x).group(1)) for x in label_filenames]


In [9]:
labels = body_part_grayscale.values()
centroids = np.array([center_of_mass(label_image == label) for label in labels])

# Round to nearest integer so that image can be indexed
centroid_coords = np.round(centroids).astype(int)

In [10]:
depths = depth_image[centroid_coords[:, 0], centroid_coords[:, 1]] / 10

In [11]:
# Points in image space, in form (x, y, z)
points_image = np.column_stack((np.fliplr(centroid_coords), depths))

In [12]:
# Convert to real coordinates
points_real = np.apply_along_axis(im.image_to_real, 1, points_image, 
                                  x_res, y_res, f_xz, f_yz)


In [13]:
part_types = ['HEAD', 'HIP', 'THIGH', 'KNEE', 'CALF', 'FOOT']

part_names, part_labels = zip(*body_part_grayscale.items())

order_l = ['L_' + x if x != 'HEAD' else x for x in part_types ]
order_r = ['R_' + x if x != 'HEAD' else x for x in part_types ]

index_l = nf.find_indices(part_names, order_l)
index_r = nf.find_indices(part_names, order_r)

In [14]:
pp.consecutive_dist(points_real[index_l])

array([73.88827535, 13.3901949 , 20.21741358, 20.4333323 , 22.61615409])

In [15]:
pp.consecutive_dist(points_real[index_r])

array([71.14739045, 20.05095822, 16.06476869, 21.02758158, 23.66292628])

In [16]:
plt.figure()

plt.imshow(depth_image)
plt.scatter(points_image[:, 0], points_image[:, 1], 
            c='w', edgecolor='k')

plt.title('Frame {}'.format(frame))

plt.show()